In [1]:
import hyperspy.api as hs
import numpy as np
from qem.image_fitting import ImageModelFitting
import cv2

%matplotlib qt

# file = '../data/STO/haadf_small.hspy'

# file = '/home/zzhang/OneDrive/code/qem/data/SRO-CTO/ADF time step .hspy'
# file = '/home/zzhang/OneDrive/data/Nico/SRO-CTO Superlattice/9-45/restore_hdf5/ADF time step .hspy'
file = '/home/zzhang/OneDrive/code/qem/examples/superlattice_adf.h5'
s = hs.load(file)
image = s.data

dx = s.axes_manager[1].scale *10 # convert nm to Angstrom
# dx = 0.72548828 # Angstrom, calculated from fft

image = image[50:-50,1000:-1000]
# image = image - 1.43e4
image = image - image.min()

# rotate the image by 45 degrees
from scipy import ndimage

# theta = 45
# rotated_image = ndimage.rotate(image, theta, reshape=False, mode='constant')

# Replace the original image with the rotated one
# image = rotated_image[:,1200:2800]

# image = image.max() - image
model=ImageModelFitting(image, dx=dx,units='A')

INFO:root:Elements: ['A', 'B', 'C'], the order is used for the atom types. Please initiate the correct elements in your system.


In [6]:
model.plot() # inspect the image

In [2]:
model.find_peaks(min_distance=10,threshold_rel=0.05) # find atomic columns by local maximum

INFO:root:Updated peak locations.


array([[  31., 1859.],
       [  82., 1860.],
       [ 184., 1859.],
       ...,
       [   0., 1540.],
       [   0., 1011.],
       [   0.,  941.]])

In [3]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/SRO-CTO/CaTiO3.cif' 
model.map_lattice(cif_file=cif_file,elements=['Ca','Ti'], reciprocal=False,region_index=0) # map the cif unitcell to the image and tile it according the local peak positions

INFO:root:Reading CIF file: /home/zzhang/OneDrive/code/qem/data/SRO-CTO/CaTiO3.cif. The elements selected are ['Ca', 'Ti'] and the elements in the CIF file are Ca4Ti4O4
/home/zzhang/OneDrive/code/qem/qem/gui_classes.py:373: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(
INFO:root:Origin selected: [ 534. 2703.]
INFO:root:Vector a selected: [102.   0.]
INFO:root:Vector b selected: [ -2. 141.]
INFO:root:In pixel: Origin: [ 534. 2703.], Vector a: [102.   0.], Vector b: [ -2. 141.]
INFO:root:In space: Origin: [ 29.72053528 150.43933868] A, Vector a: [5.67695618 0.        ] A, Vector b: [-0.11131287  7.84755707] A
INFO:root:Perfect a: [96.52161844  0.        ] pixel, Perfect b: [  0.        137.2053431] pixel by rotation of unit cell and scaling with pixel size.


AtomicColumns(lattice=Atoms(symbols='Ca884Ti884', pbc=False, cell=[59.6636962890625, 165.41091918945312, 5.462797], spacegroup_kinds=...), lattice_ref=Atoms(symbols='Ca884Ti884', pbc=False, cell=[59.6636962890625, 165.41091918945312, 5.462797], spacegroup_kinds=...), elements=['Ca', 'Ti'], tol=0, pixel_size=0.05565643310546875, reference={'origin': array([ 534., 2703.]), 'vector_a': array([96.52161844,  0.        ]), 'vector_b': array([  0.       , 137.2053431])})

In [9]:
# model.init_params()
model.refine_center_of_mass()

Refining center of mass: 100%|██████████| 1767/1767 [00:03<00:00, 475.31it/s]


{'pos_x': array([   7.3358508 ,    8.0086642 ,    9.62672259, ..., 1061.30587363,
        1060.21502877, 1059.64559154]),
 'pos_y': array([197.78648537, 111.69672723,  20.37252484, ..., 267.34136887,
        409.35592372, 548.91042626]),
 'height': array([3764.3552, 3931.1277, 3681.5725, ..., 2650.474 , 1654.1777,
        3092.3022], dtype=float32),
 'sigma': array([12.57716244, 12.57716244]),
 'background': 0.0}

In [20]:
params = model.init_params()

model.fit_voronoi(params)


Fitting cells: 100%|██████████| 1758/1758 [00:05<00:00, 309.93it/s]
INFO:root:Convergence rate for pos_x = 0.6080784245302766
INFO:root:Convergence rate for pos_y = 0.2544342021652817
INFO:root:Convergence rate for height = 0.0
INFO:root:Convergence rate for sigma = 0.0
INFO:root:Convergence reached


{'pos_x': array([  22.        ,   22.56086206,   22.71074929, ..., 1071.71074929,
        1071.71074929, 1071.71074929]),
 'pos_y': array([2845.        , 2949.02370992, 2916.        , ...,  375.        ,
         513.        ,  654.        ]),
 'height': array([2336.2231, 3298.0532, 2296.5454, ..., 1209.9805, 3254.4465,
        3261.921 ], dtype=float32),
 'sigma': array([12.57716244, 12.57716244])}

In [22]:
model.view_3d() # view the 3D model using ase of Atoms object, with coordinates refined from the image

In [19]:
model.select_atoms() # select the region where has atoms

INFO:root:Selected 1758 atoms out of 1759 atoms.


array([False,  True,  True, ...,  True,  True,  True])

In [40]:
model.plot_coordinates(10) # plot the coordinates of the selected region with element labels

In [28]:
# params = model.init_params(atom_size=0.5)
model.fit_voronoi()


Fitting cells: 100%|██████████| 1758/1758 [00:05<00:00, 304.75it/s]
INFO:root:Convergence rate for pos_x = 0.011967227807048175
INFO:root:Convergence rate for pos_y = 0.009155284069144898
INFO:root:Convergence rate for height = 0.0
INFO:root:Convergence rate for sigma = 0.0
INFO:root:Convergence reached


{'pos_x': array([  22.        ,   22.56086206,   22.71074929, ..., 1071.71074929,
        1071.71074929, 1071.71074929]),
 'pos_y': array([2845.        , 2949.02370992, 2916.        , ...,  375.        ,
         513.        ,  654.        ]),
 'height': array([2336.2231, 3298.0532, 2296.5454, ..., 1209.9805, 3254.4465,
        3261.921 ], dtype=float32),
 'sigma': array([12.57716244, 12.57716244])}

In [ ]:
import os
!export CUDA_VISIBLE_DEVICES=0
os.environ['JAX_PLATFORMS'] = 'gpu'
model.fit_background = False
params = model.init_params(atom_size=0.7)
model.fit_random_batch(params, batch_size=4000, maxiter=50, tol=5e-3, plot=False, step_size=0.01,verbose=False,num_epoch=5) # set the maxiter and tol and num_epoch to be larger for better fitting

In [31]:
model.update_coordinates()
model.update_region_crysal_analyzer()

INFO:root:The coordinates have converged.
INFO:root:Updated region 0 coordinates for crystal analyzer.


In [13]:
model.plot_fitting() # compare the fitting result with the original image

In [30]:
model.plot_scs(per_element=True,s=10) # plot the scattering cross section of the elements in the selected region, show each element separately

In [14]:
model.plot_scs_histogram() # plot the histogram of the scattering cross section of the selected region

In [41]:

model.voronoi_integration(plot=True) # integrate the scattering cross section of the selected region using voronoi tessellation

Integrating: 100%|██████████| 1758/1758 [00:04<00:00, 388.14it/s]                 


(array([ 8174.29485751, 10783.27550203,  7876.22458842, ...,
         2239.57388934,  7408.65695896, 10485.87122523]),
 <Signal2D, title: , dimensions: (|1072, 2972)>,
 <Signal2D, title: , dimensions: (|1072, 2972)>)

In [43]:
model.plot_scs_voronoi(per_element=True,s=20) # plot the scattering cross section of the selected region using voronoi tessellation

In [21]:
model.region_crysal_analyzer[0].plot_strain()

In [33]:
model.region_crysal_analyzer[0].plot_polarization(a_element='Ca', b_element='Ti',cutoff_radius=5.0,exclude_border=True,border_pixel=
100,vector_scale=50.0)

In [45]:
model.region_crysal_analyzer[0].plot_displacement(mode='local', cut_off = 5.0)

In [46]:
model.region_crysal_analyzer[0].plot_oxygen_tilt(a_type=0, o_type=2, cutoff_radius=5.0)

In [65]:
from matplotlib.collections import LineCollection
import numpy as np
import matplotlib.pyplot as plt
from qem.color import get_unique_colors

def plot_lattice_parameter(crystal, units='A'):
    """
    Plot local lattice parameters using adaptive cell origins.
    The lattice parameter is defined as the distance between neighboring origins in a and b directions.
    """
    adaptive_cells = crystal.get_origin_offset("adaptive")
    origins = np.array(list(adaptive_cells.values()))  # shape (N, 2)
    dx = crystal.dx

    # Get direction unit vectors
    a_vec = crystal.a_vector['perfect']
    b_vec = crystal.b_vector['perfect']
    a_hat = a_vec / np.linalg.norm(a_vec)
    b_hat = b_vec / np.linalg.norm(b_vec)

    lines_a = []
    values_a = []
    lines_b = []
    values_b = []
    for i, origin in enumerate(origins):
        rel = origins - origin
        # Project onto a and b directions
        proj_a = rel @ a_hat
        proj_b = rel @ b_hat
        # Find the closest neighbor in +a direction (exclude self)
        mask_a = (proj_a > 0.1) & (np.abs(rel @ b_hat) < np.linalg.norm(b_vec)/2)
        if np.any(mask_a):
            j = np.argmin(np.where(mask_a, proj_a, np.inf))
            lines_a.append([origin, origins[j]])
            values_a.append(np.linalg.norm(origins[j] - origin) * dx)
        # Find the closest neighbor in +b direction (exclude self)
        mask_b = (proj_b > 0.1) & (np.abs(rel @ a_hat) < np.linalg.norm(a_vec)/2)
        if np.any(mask_b):
            j = np.argmin(np.where(mask_b, proj_b, np.inf))
            lines_b.append([origin, origins[j]])
            values_b.append(np.linalg.norm(origins[j] - origin) * dx)

    # Atom coloring by type
    x = crystal.atomic_columns.x
    y = crystal.atomic_columns.y
    atom_types = crystal.atomic_columns.atom_types
    unique_types = np.unique(atom_types)
    color_map = {atype: color for atype, color in zip(unique_types, get_unique_colors())}
    atom_colors = [color_map[atype] for atype in atom_types]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(crystal.image, cmap="gray")
    lc_a = LineCollection(lines_a, array=np.array(values_a), cmap='Blues', linewidths=2)
    plt.gca().add_collection(lc_a)
    plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_a = plt.colorbar(lc_a)
    cbar_a.set_label(f'Lattice parameter a ({units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter a map')

    plt.subplot(1, 2, 2)
    plt.imshow(crystal.image, cmap="gray")
    lc_b = LineCollection(lines_b, array=np.array(values_b), cmap='Greens', linewidths=2)
    plt.gca().add_collection(lc_b)
    plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_b = plt.colorbar(lc_b)
    cbar_b.set_label(f'Lattice parameter b ({units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter b map')
    plt.show()

plot_lattice_parameter(model.region_crysal_analyzer[0])


In [4]:
from matplotlib.collections import LineCollection
import numpy as np
import matplotlib.pyplot as plt
from qem.color import get_unique_colors

def plot_lattice_parameter(crystal, units='A'):
    """
    Plot local lattice parameters using adaptive cell origins.
    The lattice parameter is defined as the distance between neighboring origins in a and b directions.
    """
    adaptive_cells = crystal.get_origin_offset("adaptive")
    origins = np.array(list(adaptive_cells.values()))  # shape (N, 2)
    dx = crystal.dx

    # Get direction unit vectors
    a_vec = crystal.a_vector['perfect']
    b_vec = crystal.b_vector['perfect']
    a_hat = a_vec / np.linalg.norm(a_vec)
    b_hat = b_vec / np.linalg.norm(b_vec)

    lines_a = []
    values_a = []
    lines_b = []
    values_b = []
    for i, origin in enumerate(origins):
        rel = origins - origin
        # Project onto a and b directions
        proj_a = rel @ a_hat
        proj_b = rel @ b_hat
        # Find the closest neighbor in +a direction (exclude self)
        mask_a = (proj_a > 0.1) & (np.abs(rel @ b_hat) < np.linalg.norm(b_vec)/2)
        if np.any(mask_a):
            j = np.argmin(np.where(mask_a, proj_a, np.inf))
            lines_a.append([origin, origins[j]])
            values_a.append(np.linalg.norm(origins[j] - origin) * dx)
        # Find the closest neighbor in +b direction (exclude self)
        mask_b = (proj_b > 0.1) & (np.abs(rel @ a_hat) < np.linalg.norm(a_vec)/2)
        if np.any(mask_b):
            j = np.argmin(np.where(mask_b, proj_b, np.inf))
            lines_b.append([origin, origins[j]])
            values_b.append(np.linalg.norm(origins[j] - origin) * dx)

    # Atom coloring by type
    x = crystal.atomic_columns.x
    y = crystal.atomic_columns.y
    atom_types = crystal.atomic_columns.atom_types
    unique_types = np.unique(atom_types)
    color_map = {atype: color for atype, color in zip(unique_types, get_unique_colors())}
    atom_colors = [color_map[atype] for atype in atom_types]

    plt.subplot(1, 2, 1)
    plt.imshow(crystal.image, cmap="gray")
    lc_a = LineCollection(lines_a, array=np.array(values_a), cmap='Blues', linewidths=2)
    plt.gca().add_collection(lc_a)
    # plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_a = plt.colorbar(lc_a)
    cbar_a.set_label(f'Lattice parameter a ({units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter a map')
    # set the plot to the view of the image
    plt.xlim(0, crystal.image.shape[1])
    plt.ylim(0, crystal.image.shape[0])

    plt.subplot(1, 2, 2)
    plt.imshow(crystal.image, cmap="gray")
    lc_b = LineCollection(lines_b, array=np.array(values_b), cmap='Greens', linewidths=2)
    plt.gca().add_collection(lc_b)
    # plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_b = plt.colorbar(lc_b)
    cbar_b.set_label(f'Lattice parameter b ({units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter b map')
    # set the plot to the view of the image
    plt.xlim(0, crystal.image.shape[1])
    plt.ylim(0, crystal.image.shape[0])
    plt.show()


plot_lattice_parameter(model.region_crysal_analyzer[0])


In [46]:
def plot_lattice_parameter_ab_nearest(crystal, units='A', show_lattice=False, angle_thresh=0.95, dist_min=1, dist_max_a=3, dist_max_b=3,boundary_thresh=100):
    """
    Plot local lattice parameters using all nearest neighbors in the a and b directions
    within an angular and distance cutoff.
    """
    dx = crystal.dx
    a_vec = crystal.a_vector['perfect']
    b_vec = crystal.b_vector['perfect']
    a_hat = a_vec / np.linalg.norm(a_vec)
    b_hat = b_vec / np.linalg.norm(b_vec)

    if dist_max_a is None:
        # Guess a reasonable maximum distance (e.g., 1.5 × norm of a_vec)
        dist_max_a  = 1.3 * np.linalg.norm(a_vec) * dx
    if dist_max_b is None:
        # Guess a reasonable maximum distance (e.g., 1.5 × norm of b_vec)
        dist_max_b  = 1.3 * np.linalg.norm(b_vec) * dx

    lines_a, values_a = [], []
    lines_b, values_b = [], []

    for i, coord in enumerate(crystal.peak_positions):
        x,y = coord
        if (
            (x < boundary_thresh) or (x > crystal.image.shape[1] - boundary_thresh) or
            (y < boundary_thresh) or (y > crystal.image.shape[0] - boundary_thresh)
        ):
            continue  # Skip if near boundary
        rel = crystal.peak_positions - coord
        dists = np.linalg.norm(rel, axis=1) * dx
        # Exclude self
        valid_a  = (dists > dist_min) & (dists < dist_max_a)
        valid_b = (dists > dist_min) & (dists < dist_max_b)

        # For a direction
        cos_a = np.abs((rel @ a_hat) / (np.linalg.norm(rel, axis=1) + 1e-12))
        mask_a = valid_a & (cos_a > angle_thresh)
        for j in np.where(mask_a)[0]:
            lines_a.append([coord, crystal.peak_positions[j]])
            values_a.append(dists[j])

        # For b direction
        cos_b = np.abs((rel @ b_hat) / (np.linalg.norm(rel, axis=1) + 1e-12))
        mask_b = valid_b & (cos_b > angle_thresh)
        for j in np.where(mask_b)[0]:
            lines_b.append([coord, crystal.peak_positions[j]])
            values_b.append(dists[j])

    # Atom coloring by type
    x = crystal.atomic_columns.x
    y = crystal.atomic_columns.y
    atom_types = crystal.atomic_columns.atom_types
    unique_types = np.unique(atom_types)
    color_map = {atype: color for atype, color in zip(unique_types, get_unique_colors())}
    atom_colors = [color_map[atype] for atype in atom_types]

    plt.subplot(1, 2, 1)
    plt.imshow(crystal.image, cmap="gray")
    lc_a = LineCollection(lines_a, array=np.array(values_a), cmap='Blues', linewidths=2)
    plt.gca().add_collection(lc_a)
    if show_lattice:
        plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_a = plt.colorbar(lc_a)
    cbar_a.set_label(f'Lattice parameter a (nearest, {units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter a (nearest) map')
    plt.xlim(0, crystal.image.shape[1])
    plt.ylim(0, crystal.image.shape[0])

    plt.subplot(1, 2, 2)
    plt.imshow(crystal.image, cmap="gray")
    lc_b = LineCollection(lines_b, array=np.array(values_b), cmap='Greens', linewidths=2)
    plt.gca().add_collection(lc_b)
    if show_lattice:
        plt.scatter(x, y, c=atom_colors, s=15, edgecolor='k', linewidth=0.5)
    cbar_b = plt.colorbar(lc_b)
    cbar_b.set_label(f'Lattice parameter b (nearest, {units})')
    plt.axis('off')
    if hasattr(crystal, 'scalebar'):
        plt.gca().add_artist(crystal.scalebar)
    plt.title('Lattice parameter b (nearest) map')
    plt.xlim(0, crystal.image.shape[1])
    plt.ylim(0, crystal.image.shape[0])
    plt.show()

plot_lattice_parameter_ab_nearest(model.region_crysal_analyzer[0])
